In [ ]:
import chromedriver_autoinstaller
chromedriver_autoinstaller.install()
from selenium import webdriver

# -*- coding: UTF-8 -*-
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import sys 
import pandas as pd
import requests
import time

In [ ]:
webdriver_options = webdriver.ChromeOptions()
webdriver_options .add_argument('headless')
web_driver = webdriver.Chrome(options=webdriver_options)
url = "https://map.kakao.com/"
web_driver.get(url)
time.sleep(3)

## 장소에 대한 전화번호, 운영 시간 크롤링

In [ ]:
def getPhoneAndOpening_hours(placeName):
    if not placeName:
        return [None,None]
    search_input = WebDriverWait(web_driver, 20).until(EC.visibility_of_element_located((By.XPATH,'//*[@id="search.keyword.query"]')))
    search_input.click()
    search_input.clear()
    search_input.send_keys(placeName)
    print(placeName,end=" ")
    search_input.send_keys("\n")
    # web_driver.implicitly_wait(10)
    html = web_driver.page_source
    soup = BeautifulSoup(html,'html.parser')
    soup = soup.find('ul',class_='placelist').find('li',class_="PlaceItem clickArea")
    if soup:
        phone = soup.find('span',class_='phone').text
        opening_hours=soup.find('p',class_='periodWarp').find('a').text
        return [phone,opening_hours]
    else:
        return [None,None]
        

## 데이터 저장

In [ ]:
def save(rows):
    df= pd.DataFrame(rows,columns=df_cols)
    try:
        data = pd.read_csv("장애인편의시설.csv", encoding='utf-8-sig')
        df2 = pd.DataFrame(data)
        new_df = pd.concat([df, df2])
        new_df.drop_duplicates()
        new_df.to_csv("장애인편의시설.csv",index=False, encoding="utf-8-sig")
    except:
        df.to_csv("장애인편의시설.csv",index=False, encoding="utf-8-sig")
        data = pd.read_csv("장애인편의시설.csv", encoding='utf-8-sig')
        new_df = pd.DataFrame(data)

# [특정 장애인편의시설 내 가구표 조회]상세기능명세 api 호출

In [ ]:
serviceKey = 'YOUR_KEY'
def getFacList(wfcltId):
    url = 'http://apis.data.go.kr/B554287/DisabledPersonConvenientFacility/getFacInfoOpenApiJpEvalInfoList'
    global serviceKey
    params = {'serviceKey' : serviceKey, 'wfcltId' : wfcltId}
    response = requests.get(url, params=params)
    soup = BeautifulSoup(response.text,'lxml-xml')
    evalInfo=None
    if soup.find("servList"):
        soup = soup.find("servList")
        evalInfo = "API Request Fail"
        if soup.find("evalInfo"):
            evalInfo = soup.find("evalInfo").text
    return evalInfo

## api 호출 및 데이터 저장

In [ ]:
url = 'http://apis.data.go.kr/B554287/DisabledPersonConvenientFacility/getDisConvFaclList'
detail_url = 'http://apis.data.go.kr/B554287/DisabledPersonConvenientFacility/getFacInfoOpenApiJpEvalInfoList'
pageNo=list(range(265,271))

df_cols=["faclNm","faclTyCd","lcMnad","faclLat","faclLng","salStaDivCd","wfcltId","phone","opening_hours","evalInfo"]


for page in pageNo:
    global serviceKey
    params = {'serviceKey' : serviceKey, 'pageNo' : page,'numOfRows':500 }
    response = requests.get(url, params=params)
    soup = BeautifulSoup(response.text,'lxml-xml')
    items = soup.find_all("servList")
    rows=[]
    i=0
    for item in items:
        if item is None:
            pass
        faclNm=item.find("faclNm").get_text() if item.find("faclNm") is not None else None
        faclTyCd=item.find("faclTyCd").get_text() if item.find("faclTyCd") is not None else None
        lcMnad=item.find("lcMnad").get_text() if item.find("lcMnad") is not None else None
        faclLat=item.find("faclLat").get_text() if item.find("faclLat") is not None else None
        faclLng=item.find("faclLng").get_text() if item.find("faclLng") is not None else None
        salStaDivCd=item.find("salStaDivCd").get_text() if item.find("salStaDivCd") is not None else None
        wfcltId=item.find("wfcltId").get_text() if item.find("wfcltId") is not None else None
        flag = True
        while flag:
        # 전화번호, 영업시간
            try:
                result = getPhoneAndOpening_hours(faclNm)
                flag=False
            except:
                #해당 url로 이동
                kakao_url = "https://map.kakao.com/" 
                web_driver.get(kakao_url)
                web_driver.implicitly_wait(20)
                flag=True
        # 시설 기구표
        evalInfo = getFacList(wfcltId)
        
        row={"faclNm":faclNm, "faclTyCd":faclTyCd,
                    "lcMnad":lcMnad,"faclLat":faclLat,
                    "faclLng":faclLng,"salStaDivCd":salStaDivCd,
                    "wfcltId":wfcltId,"phone":result[0],
                    "opening_hours":result[1],"evalInfo":evalInfo}
        print((page-1)*500+i)
        i+=1
        rows.append(row)
    save(rows)
    print(page," 완료")
